COMP 215 - LAB 2 Records (NEO)
----------------
#### Name: Arshpreet singh
#### Date: 2024/01/15

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [3]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [4]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2024-01-21 <class 'datetime.date'>
Today is: 2024-01-21


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [5]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

today = '2023-01-09'  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:5])

13 Near Earth Objects found for 2023-01-09
[{'absolute_magnitude_h': 19.63,
  'close_approach_data': [{'close_approach_date': '2023-01-09',
                           'close_approach_date_full': '2023-Jan-09 13:59',
                           'epoch_date_close_approach': 1673272740000,
                           'miss_distance': {'astronomical': '0.0725378623',
                                             'kilometers': '10851509.694433301',
                                             'lunar': '28.2172284347',
                                             'miles': '6742815.4556135138'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '32387.3479203461',
                                                 'kilometers_per_second': '8.9964855334',
                                                 'miles_per_hour': '20124.2384897284'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 2312.1980468998,
     

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [6]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [7]:
# Ex. 1 your code here

n_hazard = len([item for item in neos if item['is_potentially_hazardous_asteroid']])
print(f'{n_hazard} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [8]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 19.63,
 'close_approach_data': [{'close_approach_date': '1900-10-30',
                          'close_approach_date_full': '1900-Oct-30 21:15',
                          'epoch_date_close_approach': -2182819500000,
                          'miss_distance': {'astronomical': '0.4183488933',
                                            'kilometers': '62584103.354537271',
                                            'lunar': '162.7377194937',
                                            'miles': '38887958.5659094998'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '30027.9671888418',
                                                'kilometers_per_second': '8.3411019969',
                                                'miles_per_hour': '18658.2110568667'}},
                         {'close_approach_date': '1901-06-26',
                          'close_approach_date_full': '1901-Jun-26 20:27

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [9]:
# Ex. 2 your code here

close_approach_data = []

for i in hazards:
   close_approach_data.append(i['close_approach_data'][0])

def closest_approach(dta):
  ''' This function takes the close approach data as input and returns a tuple containing closest date of approach and miss distance in kilometers'''
  dist = []
  closest_data =[]
  mydict ={}

  for j in dta:
     mydict[(float(j['miss_distance']['kilometers']))] = j['close_approach_date']
     dist.append((float(j['miss_distance']['kilometers'])))

  closest_data.append(min(dist))
  closest_data.append(mydict[min(dist)])

  return tuple(closest_data)

assert closest_approach([{'close_approach_date': '2193-09-16',
                          'close_approach_date_full': '2193-Sep-16 23:41',
                          'epoch_date_close_approach': 7059656460000,
                          'miss_distance': {'astronomical': '0.4305039114',
                                            'kilometers': '64402468.172108718',
                                            'lunar': '167.4660215346',
                                            'miles': '40017838.0703395884'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '27939.5929345009',
                                                'kilometers_per_second': '7.7609980374',
                                                'miles_per_hour': '17360.5765097736'}},
                         {'close_approach_date': '2194-07-26',
                          'close_approach_date_full': '2194-Jul-26 01:01',
                          'epoch_date_close_approach': 7086618060000,
                          'miss_distance': {'astronomical': '0.4918243047',
                                            'kilometers': '73575868.397350989',
                                            'lunar': '191.3196545283',
                                            'miles': '45717924.6537780882'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '81015.8441679468',
                                                'kilometers_per_second': '22.5044011578',
                                                'miles_per_hour': '50340.0949498001'}},]) == (64402468.172108718, '2193-09-16')


## Challenge - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [31]:
# Ex. 3 (challenge) your code here

api_key = 'DEMO_KEY'
start_date = datetime.date.today()
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date=&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)
pprint(data)
for i in (data['near_earth_objects']['2024-01-21']):
  for j in i['close_approach_data']:
    j['close_approach_date']
    j['miss_distance']['kilometers']

{'element_count': 105,
 'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-28&end_date=2024-02-04&detailed=false&api_key=DEMO_KEY',
           'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-14&end_date=2024-01-21&detailed=false&api_key=DEMO_KEY',
           'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-21&end_date=2024-01-28&detailed=false&api_key=DEMO_KEY'},
 'near_earth_objects': {'2024-01-21': [{'absolute_magnitude_h': 16.17,
                                        'close_approach_data': [{'close_approach_date': '2024-01-21',
                                                                 'close_approach_date_full': '2024-Jan-21 '
                                                                                             '02:40',
                                                                 'epoch_date_close_approach': 1705804800000,
                                                                 'miss_distance':

KeyError: 'close+'